In [37]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

From https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html

In [38]:
'''Visualization of the filters of VGG16, via gradient ascent in input space.

This script can run on CPU in a few minutes.

Results example: http://i.imgur.com/4nj4KjN.jpg
'''
from __future__ import print_function

from scipy.misc import imsave
import numpy as np
import time
from keras.applications import vgg16
from keras import backend as K

import pickle
import h5py

# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv1'

'Visualization of the filters of VGG16, via gradient ascent in input space.\n\nThis script can run on CPU in a few minutes.\n\nResults example: http://i.imgur.com/4nj4KjN.jpg\n'

In [39]:
# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

## To calculate number of parameters (i.e., the Param # column) for a layer
## $c_i$ number of channels for input to layer
## $F$: filter size is FxFx$c_i$.  $F$ seems to be 3
## $c_o$ is number of channels for output of layer
## $count_{parameters} = (F * F * c_i +1) * c_o$
### The "+1" above is because there is 1 bias per output channel

In [40]:
# build the VGG16 network with ImageNet weights
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

58613760/58889256 [============================>.] - ETA: 0s  

In [41]:
kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

Processing filter 0
Current loss value: 12.1301
Current loss value: 23.8688
Current loss value: 45.7799
Current loss value: 70.4341
Current loss value: 95.4459
Current loss value: 128.848
Current loss value: 158.129
Current loss value: 188.438
Current loss value: 215.174
Current loss value: 242.597
Current loss value: 264.05
Current loss value: 290.749
Current loss value: 314.258
Current loss value: 337.76
Current loss value: 361.907
Current loss value: 382.714
Current loss value: 406.412
Current loss value: 433.748
Current loss value: 454.192
Current loss value: 478.821
Filter 0 processed in 22s
Processing filter 1
Current loss value: 0.0190822
Current loss value: 10.206
Current loss value: 28.1641
Current loss value: 43.7796
Current loss value: 61.8538
Current loss value: 95.6591
Current loss value: 127.184
Current loss value: 176.197
Current loss value: 236.036
Current loss value: 291.572
Current loss value: 371.212
Current loss value: 459.693
Current loss value: 547.434
Current los

## Note: %store stores in ~/.ipython/profile_defaults/db/autorestore/<variable name>
### Be sure the home directory is on an AWS volume that is saved (i.e., the directory for Jupyter notebooks may be kept on a separate volume from ~

In [42]:
%store kept_filters

Stored 'kept_filters' (list)


## Store kept_filters on disk so don't have to re-compute

In [43]:
pkl_file = "kept_filters.pkl"

In [44]:
with open(pkl_file, "wb") as fh:
    pickle.dump(kept_filters, fh)


## Recover kept_filters from disk
### Can also try: store -r kept_filters

In [45]:
with open(pkl_file, "rb") as fh:
    r_p_kept_filters = pickle.load(fh)

In [51]:
type(r_p_kept_filters)

list

In [47]:
kept_filters=r_p_kept_filters

In [1]:
%store -r kept_filters

In [2]:
type(kept_filters)

list

In [ ]:
# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

# save the result to disk
imsave('stitched_filters_%dx%d.png' % (n, n), stitched_filters)